In [1]:
import pandas as pd
import numpy as np
from openpyxl import Workbook as wb

In [2]:
July_SEs_RW = pd.read_csv("data/PEAL - Standards Evaluation - July 2024_Rwanda.csv", encoding='cp1252', low_memory=False)

In [3]:
July_SEs_RW.head(1)

,SubmissionDate,starttime,endtime,text_audit,ev_date,contractor_name,cluster_t,village_t,standards_report,hhid,...,farm_pract_seen_nurs,liquid_manure_nursery,organic_pesticides_nursery,nursery_watered,veg_varieties_planted_in_nursery,village_instance,instanceID,instanceName,formdef_version,KEY
0,22/07/2024 18:10,21/07/2024 10:26,21/07/2024 10:47,https://rtvuganda.surveycto.com/view/submissio...,21/07/2024,Manirakiza Fred,Mwiyanike,Gitebe,Household,RW-NYA-GIT-AUG-M-140650,...,NaN,NaN,NaN,NaN,NaN,Gitebe,uuid:2899f971-2d5a-43c6-9f2a-67d126ea2ed0,Household : Gitebe,2407212132,uuid:2899f971-2d5a-43c6-9f2a-67d126ea2ed0


In [4]:
HHLevel =July_SEs_RW[July_SEs_RW['standards_report'] == 'Household'].copy()

In [5]:
HHLevel['Water control Practice'] = np.where(HHLevel['water_mgt_methods_num'] > 0, 1,0)
HHLevel['Soil Management Practice'] = np.where(HHLevel['soil_mgt_methods_num'] > 1, 1,0)
HHLevel['KAP Livestock'] = np.where(HHLevel['kap_livestock_num'] > 3, 1,0)
HHLevel['KAP Agriculture'] = np.where(HHLevel['kap_farm_practices_num'] > 3, 1,0)
HHLevel['3 Composts'] = np.where(HHLevel['composts_num'] > 2, 1,0)
HHLevel['Vegetable gardening Practices'] = np.where(HHLevel['vegetable_farm_mode_num'] > 0, 1,0)

In [6]:
HHLevel[['Water control Practice', 'Soil Management Practice', 
     'Vegetable gardening Practices', 'KAP Livestock', 'KAP Agriculture', '3 Composts']].head(1)

,Water control Practice,Soil Management Practice,Vegetable gardening Practices,KAP Livestock,KAP Agriculture,3 Composts
0,1,1,1,0,1,0


In [7]:
HHLevel['Compost Compliance'] =  HHLevel[['compost_heap_base', 'compost_standard_size', 'windrow_size_dimension',
                                  'heap_covered', 'compost_demarcated', 'forms_non_bio', 
                                          'composts_watered_regularly', 'composts_manure_usuage']].mean(axis = 1)

In [8]:
HHLevel['Drying Method Num'] = HHLevel[['drying_method_1','drying_method_2',
                                    'drying_method_3']].sum(axis = 1)

In [9]:
HHLevel['Drying Method'] = np.where(HHLevel['Drying Method Num'] > 1,1,0)

In [10]:
HHLevel['Post harvest handling'] = HHLevel[['Drying Method', 'sorting_grading_done', 
                                     'post_harvest_sys_present']].mean(axis = 1)

In [11]:
HHLevel['Agric Compliance'] = HHLevel[['Compost Compliance', 'Post harvest handling', 'Water control Practice',
                               'Soil Management Practice', 'Vegetable gardening Practices', 
                               'organic_manure_received']].mean(axis = 1)

In [12]:
WASH = ['latrine_present', 'latrine_cover_present', 'latrine_floor_sealed',
                              'latrine_door_present', 'latrine_walls_sealed', 'latrine_roofleak_proof',
                              'latrine_environ_free_debris', 'latri_environ_free_feaces', 'tippy_tap_present',
                              'tippy_fill_water', 'soap_ash_present', 'tippy_tap_wet_area', 'cloth_hangline',
                              'kitchen_present', 'kitchen_ventilated', 'kitchen_clean', 'bathroom_present',
                              'bath_drainage_present', 'bathroom_clean', 'compound_clean', 'double_dishrack',
                              'non_bio_waste_properly_sorted']

In [13]:
HHLevel[WASH] = HHLevel[WASH].apply(pd.to_numeric, errors='coerce')

In [14]:
HHLevel['WASH Compliance'] = HHLevel[WASH].mean(axis = 1)

In [15]:
Livestock = ['gs_pen_constr', 'gs_pen_constr', 'gs_droppings_composted', 
             'pg_sty_constr', 'pg_sty_clean', 'pg_droppings_composted',
            'cow_kraal_constr', 'kraal_clean', 'cow_droppings_composted']

In [16]:
HHLevel['Livestock Compliance'] = HHLevel[Livestock].mean(axis = 1)

In [17]:
HHLevel['Overall Compliance'] = HHLevel[['Agric Compliance', 'WASH Compliance', 
                                 'Livestock Compliance', 'vlsa_membership']].mean(axis = 1)

In [18]:
HHLevel['NPS Classified'] = np.where(HHLevel['overall_nps'] < 70, 'Detractor', 
                                 np.where(HHLevel['overall_nps'] > 80, 'Promoter','Passive'))

In [19]:
Village_NPS = HHLevel.groupby(['district_1', 'cluster_t', 'village_t'])['NPS Classified'].value_counts().unstack(fill_value=0)

Village_NPS['Total'] = Village_NPS.sum(axis=1)

Village_NPS['Detractors(%)'] = (Village_NPS['Detractor'] / Village_NPS['Total']) * 100
Village_NPS['Passives(%)'] = (Village_NPS['Passive'] / Village_NPS['Total']) * 100
Village_NPS['Promoters(%)'] = (Village_NPS['Promoter'] / Village_NPS['Total']) * 100

Village_NPS['Overall NPS (%)'] = Village_NPS['Promoters(%)'] - Village_NPS['Detractors(%)']

Village_NPS = Village_NPS.reset_index()[
    ['district_1', 'cluster_t', 'village_t', 'Detractors(%)', 'Passives(%)', 'Promoters(%)', 'Overall NPS (%)']
]
Village_NPS.to_excel('RWVillage_NPS.xlsx', index=False)

In [20]:
Cluster_NPS = HHLevel.groupby(['district_1', 'cluster_t'])['NPS Classified'].value_counts().unstack(fill_value=0)

Cluster_NPS ['Total'] = Cluster_NPS .sum(axis=1)

Cluster_NPS ['Detractors(%)'] = (Cluster_NPS ['Detractor'] / Cluster_NPS ['Total']) * 100
Cluster_NPS ['Passives(%)'] = (Cluster_NPS ['Passive'] / Cluster_NPS ['Total']) * 100
Cluster_NPS ['Promoters(%)'] = (Cluster_NPS ['Promoter'] / Cluster_NPS ['Total']) * 100

Cluster_NPS ['Overall NPS (%)'] = Cluster_NPS ['Promoters(%)'] - Cluster_NPS ['Detractors(%)']

Cluster_NPS  = Cluster_NPS .reset_index()[
    ['district_1', 'cluster_t', 'Detractors(%)', 'Passives(%)', 'Promoters(%)', 'Overall NPS (%)']
]

Cluster_NPS .to_excel('RWNPSCluster.xlsx', index=False)

In [21]:
HHLevel['VSLACommitte'] = np.where(HHLevel['vsla_committee_satification_sc'] < 70, 'Detractor', 
                                 np.where(HHLevel['vsla_committee_satification_sc'] > 80, 'Promoter','Passive'))

In [22]:
NPS_table = HHLevel.groupby(['district_1', 'cluster_t', 'village_t'])['VSLACommitte'].value_counts().unstack(fill_value=0)

# Add a total count column
NPS_table['Total'] = NPS_table.sum(axis=1)

# Calculate percentages for each category, handling missing columns gracefully
NPS_table['Detractors(%)'] = (NPS_table.get('Detractor', 0) / NPS_table['Total']) * 100
NPS_table['Passives(%)'] = (NPS_table.get('Passive', 0) / NPS_table['Total']) * 100
NPS_table['Promoters(%)'] = (NPS_table.get('Promoter', 0) / NPS_table['Total']) * 100

In [23]:
HHLevel['Agric Champions'] = np.where(HHLevel[ 'agric_champion_score'] < 7, 'Detractor', 
                                 np.where(HHLevel[ 'agric_champion_score'] > 8, 'Promoter','Passive'))

In [24]:
NPS_table = HHLevel.groupby(['district_1', 'cluster_t', 'village_t'])['Agric Champions'].value_counts().unstack(fill_value=0)

NPS_table['Total'] = NPS_table.sum(axis=1)

NPS_table['Detractors(%)'] = (NPS_table.get('Detractor', 0) / NPS_table['Total']) * 100
NPS_table['Passives(%)'] = (NPS_table.get('Passive', 0) / NPS_table['Total']) * 100
NPS_table['Promoters(%)'] = (NPS_table.get('Promoter', 0) / NPS_table['Total']) * 100


In [25]:
HHLevel['WASH Champions'] = np.where(HHLevel['wash_champion_score'] < 7, 'Detractor', 
                                 np.where(HHLevel['wash_champion_score'] > 8, 'Promoter','Passive'))

In [26]:
NPS_table = HHLevel.groupby(['district_1', 'cluster_t', 'village_t'])['WASH Champions'].value_counts().unstack(fill_value=0)

NPS_table['Total'] = NPS_table.sum(axis=1)

NPS_table['Detractors(%)'] = (NPS_table.get('Detractor', 0) / NPS_table['Total']) * 100
NPS_table['Passives(%)'] = (NPS_table.get('Passive', 0) / NPS_table['Total']) * 100
NPS_table['Promoters(%)'] = (NPS_table.get('Promoter', 0) / NPS_table['Total']) * 100

In [27]:
Village_Scores = HHLevel.groupby(['district_1',  'cluster_t',  'village_t', 'ev_month'])[['Overall Compliance', 'Agric Compliance', 
                                                                                          'WASH Compliance', 
                            'Livestock Compliance', 'vlsa_membership', 'Compost Compliance', 'compost_heap_base', 
                                'compost_standard_size', 'windrow_size_dimension',
                                  'heap_covered', 'compost_demarcated', 'forms_non_bio', 
                                          'composts_watered_regularly', 'composts_manure_usuage', 'organic_manure_received', 
                            'Drying Method', 'sorting_grading_done', 
                                     'post_harvest_sys_present',
                            'Post harvest handling', 'Water control Practice',
                            'Soil Management Practice', 'Vegetable gardening Practices', 
                            'latrine_present', 'latrine_cover_present', 'latrine_floor_sealed',
                              'latrine_door_present', 'latrine_walls_sealed', 'latrine_roofleak_proof',
                              'latrine_environ_free_debris', 'latri_environ_free_feaces', 'tippy_tap_present',
                              'tippy_fill_water', 'soap_ash_present', 'tippy_tap_wet_area', 'cloth_hangline',
                              'kitchen_present', 'kitchen_ventilated', 'kitchen_clean', 'bathroom_present',
                              'bath_drainage_present', 'bathroom_clean', 'compound_clean', 'double_dishrack',
                              'non_bio_waste_properly_sorted', 'livestock_present_1', 'livestock_present_2', 
                              'gs_pen_constr', 'gs_pen_constr', 'gs_droppings_composted', 'livestock_present_3', 
                            'pg_sty_constr', 'pg_sty_clean', 'pg_droppings_composted', 'livestock_present_4', 'cow_kraal_constr', 'kraal_clean', 
                            'cow_droppings_composted', 'KAP Livestock', 
                            'KAP Agriculture']].mean(numeric_only = True).reset_index()
Village_Scores = Village_Scores.sort_values(
    by=['district_1', 'cluster_t', 'Overall Compliance'], 
    ascending=[True, True, False]
)

In [28]:
Village_Scores.to_excel('Village_Scores.xlsx', index = False)

In [29]:
Cluster_Scores = HHLevel.groupby(['district_1',  'cluster_t', 'ev_month'])[['Overall Compliance', 'Agric Compliance', 
                                                                                          'WASH Compliance', 
                            'Livestock Compliance', 'vlsa_membership', 'Compost Compliance', 'compost_heap_base', 
                                'compost_standard_size', 'windrow_size_dimension',
                                  'heap_covered', 'compost_demarcated', 'forms_non_bio', 
                                          'composts_watered_regularly', 'composts_manure_usuage', 'organic_manure_received', 
                            'Drying Method', 'sorting_grading_done', 
                                     'post_harvest_sys_present',
                            'Post harvest handling', 'Water control Practice',
                            'Soil Management Practice', 'Vegetable gardening Practices', 
                            'latrine_present', 'latrine_cover_present', 'latrine_floor_sealed',
                              'latrine_door_present', 'latrine_walls_sealed', 'latrine_roofleak_proof',
                              'latrine_environ_free_debris', 'latri_environ_free_feaces', 'tippy_tap_present',
                              'tippy_fill_water', 'soap_ash_present', 'tippy_tap_wet_area', 'cloth_hangline',
                              'kitchen_present', 'kitchen_ventilated', 'kitchen_clean', 'bathroom_present',
                              'bath_drainage_present', 'bathroom_clean', 'compound_clean', 'double_dishrack',
                              'non_bio_waste_properly_sorted', 'livestock_present_1', 'livestock_present_2', 
                              'gs_pen_constr', 'gs_pen_constr', 'gs_droppings_composted', 'livestock_present_3', 
                            'pg_sty_constr', 'pg_sty_clean', 'pg_droppings_composted', 'livestock_present_4', 'cow_kraal_constr', 'kraal_clean', 
                            'cow_droppings_composted', 'KAP Livestock', 
                            'KAP Agriculture']].mean(numeric_only = True).reset_index()

In [30]:
Cluster_Scores.to_excel('RWCluster_Scores.xlsx', index = False)

In [31]:
Cluster_ScoresA = Cluster_Scores.applymap(lambda x: x * 100 if isinstance(x, (int, float)) else x)

/var/folders/8v/g75zp59111n27nbw3_c9ypxh0000gn/T/ipykernel_92336/2332091296.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  Cluster_ScoresA = Cluster_Scores.applymap(lambda x: x * 100 if isinstance(x, (int, float)) else x)


In [32]:
Cluster_ScoresA.to_excel('RWCluster_ScoresA.xlsx', index = False)

In [33]:
Village_ScoresA = Village_Scores.applymap(lambda x: x * 100 if isinstance(x, (int, float)) else x)

/var/folders/8v/g75zp59111n27nbw3_c9ypxh0000gn/T/ipykernel_92336/3433828518.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  Village_ScoresA = Village_Scores.applymap(lambda x: x * 100 if isinstance(x, (int, float)) else x)


In [34]:
Village_ScoresA.to_excel('RWVillage_ScoresA.xlsx', index = False)

In [35]:
Village_ScoresA.to_csv('RWVillage_ScoresA.csv', index = False)

In [36]:
def determine_conclusion(score):
    if score < 70:
        return "Off-Track - Issue of Missing Program Targets"
    elif 70 <= score <= 79:
        return "At Risk - Below Program Targets with Major Areas to Address"
    elif 80 <= score <= 89:
        return "On Track - Near Program Targets with Minor Areas to Address"
    else:
        return "On Track - Meet or Exceed Program Targets"

# Apply the function to the "Overall score" column to create a new "Conclusions" column
Village_ScoresA['Overall Compliance Comment'] = Village_ScoresA['Overall Compliance'].apply(determine_conclusion)
Village_ScoresA['Agric Compliance Comment'] = Village_ScoresA['Agric Compliance'].apply(determine_conclusion)
Village_ScoresA['WASH Compliance Comment'] = Village_ScoresA['WASH Compliance'].apply(determine_conclusion)
Village_ScoresA['Livestock Compliance Comment'] = Village_ScoresA['Livestock Compliance'].apply(determine_conclusion)
Village_ScoresA['VSLA Participation Comment'] = Village_ScoresA['vlsa_membership'].apply(determine_conclusion)

# Create a new DataFrame with the "Overall score" and "Conclusions" columns
Conclusions = Village_ScoresA[['district_1',  'cluster_t', 'village_t', 'ev_month','Overall Compliance Comment',
                              'Agric Compliance Comment', 'WASH Compliance Comment', 'Livestock Compliance Comment',
                              'VSLA Participation Comment']]

Conclusions.to_excel('RWConclusions.xlsx', index = False)

Village SEs

In [37]:
VLevel =July_SEs_RW[July_SEs_RW['standards_report'] == 'Village'].copy()

In [38]:
VLevel.head(1)

,SubmissionDate,starttime,endtime,text_audit,ev_date,contractor_name,cluster_t,village_t,standards_report,hhid,...,farm_pract_seen_nurs,liquid_manure_nursery,organic_pesticides_nursery,nursery_watered,veg_varieties_planted_in_nursery,village_instance,instanceID,instanceName,formdef_version,KEY
0,21/07/2024 21:09,17/07/2024 20:24,21/07/2024 21:09,https://rtvuganda.surveycto.com/view/submissio...,21/07/2024,Bankudiye Charlotte,Rwantobo,Karambi,Village,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,uuid:bc821fec-8be7-46d3-9832-605bd883a261,Village :,2407162058,uuid:bc821fec-8be7-46d3-9832-605bd883a261


In [64]:
VLevel_SEs = VLevel.groupby(['cluster_t',  'village_t'])[['water_sources_num', 'wuc1', 'wuc1_fetch_records', 'wuc1_contribution_records', 'water_flow1', 
                                                                         'source_sanitary_standards1', 'source_structural_standards1', 'source_demarcation_1', 'pooling_clear_1', 
                                                                         'time_spent_reduce1', 'vslas_num', 'mems_active_joint', 'contr_documentation_joint', 'keep_record_members_joint', 
                                                                         'keep_record_loans_expense_joint', 'meet_regular_joint', 'om_balance_costs_joint', 'vsla_goals_documented_joint', 
                                                                         'active_committee_func_joint', 'committee_positions_filled_joint', 'loan_repayments_tracked_join', 'loaning_intrest_joint',
                                                                         'mems_active_youth', 'contr_documentation_youth', 'keep_record_members_youth', 
                                                                         'keep_record_loans_expense_youth', 'meet_regular_youth', 'om_balance_costs_youth', 'vsla_goals_documented_youth', 
                                                                         'active_committee_func_youth', 'committee_positions_filled_youth', 'loan_repayments_tracked_youth', 'loaning_intrest_youth',
                                                                         'mems_active_women', 'contr_documentation_women', 'keep_record_members_women', 
                                                                         'keep_record_loans_expense_women', 'meet_regular_women', 'om_balance_costs_women', 'vsla_goals_documented_women', 
                                                                         'active_committee_func_women', 'committee_positions_filled_women', 'loan_repayments_tracked_women', 'loaning_intrest_women',
                                                                         'demo_cared', 'farm_pract_seen', 'organic_manure_demo', 'organic_pesticides_demo', 'liquid_manure_demo',
                                                                         'nursery_shade_constr', 'farm_pract_seen_nurs', 'liquid_manure_nursery', 
                                                                         'organic_pesticides_nursery', 'nursery_watered']].mean(numeric_only = True).reset_index()

VLevel_SEs.fillna("-", inplace=True)

VLevel_SEsA = VLevel_SEs.applymap(lambda x: x * 100 if isinstance(x, (int, float)) else x)

/var/folders/8v/g75zp59111n27nbw3_c9ypxh0000gn/T/ipykernel_80624/3939620111.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  VLevel_SEs.fillna("-", inplace=True)
/var/folders/8v/g75zp59111n27nbw3_c9ypxh0000gn/T/ipykernel_80624/3939620111.py:18: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  VLevel_SEsA = VLevel_SEs.applymap(lambda x: x * 100 if isinstance(x, (int, float)) else x)


,district_1,cluster_t,village_t,ev_month,water_sources_num,wuc1,wuc1_fetch_records,wuc1_contribution_records,water_flow1,source_sanitary_standards1,...,demo_cared,farm_pract_seen,organic_manure_demo,organic_pesticides_demo,liquid_manure_demo,nursery_shade_constr,farm_pract_seen_nurs,liquid_manure_nursery,organic_pesticides_nursery,nursery_watered


In [65]:
VLevel_SEs.head()

,cluster_t,village_t,water_sources_num,wuc1,wuc1_fetch_records,wuc1_contribution_records,water_flow1,source_sanitary_standards1,source_structural_standards1,source_demarcation_1,...,demo_cared,farm_pract_seen,organic_manure_demo,organic_pesticides_demo,liquid_manure_demo,nursery_shade_constr,farm_pract_seen_nurs,liquid_manure_nursery,organic_pesticides_nursery,nursery_watered
0,Mwiyanike,Gitebe,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
1,Mwiyanike,Kayanza,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
2,Mwiyanike,Kivuruga,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
3,Mwiyanike,Mucundebo,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
4,Mwiyanike,Musaraba,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-


Series([], Name: water_sources_num, dtype: float64)

0      NaN
16     NaN
32     NaN
48     NaN
64     NaN
91     NaN
96     NaN
115    NaN
128    NaN
138    NaN
150    NaN
154    NaN
173    NaN
190    NaN
202    NaN
222    NaN
Name: district_1, dtype: object

Series([], Name: water_sources_num, dtype: float64)

In [66]:
VLevel_SEs.head()

,cluster_t,village_t,water_sources_num,wuc1,wuc1_fetch_records,wuc1_contribution_records,water_flow1,source_sanitary_standards1,source_structural_standards1,source_demarcation_1,...,demo_cared,farm_pract_seen,organic_manure_demo,organic_pesticides_demo,liquid_manure_demo,nursery_shade_constr,farm_pract_seen_nurs,liquid_manure_nursery,organic_pesticides_nursery,nursery_watered
0,Mwiyanike,Gitebe,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
1,Mwiyanike,Kayanza,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
2,Mwiyanike,Kivuruga,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
3,Mwiyanike,Mucundebo,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
4,Mwiyanike,Musaraba,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-


In [68]:
VLevel_SEsA.to_excel('RWCommunitySEs.xlsx', index = False)

In [42]:
AgricTr_Cols = ['agric_trainings_1', 'agric_trainings_2', 'agric_trainings_3', 
                'agric_trainings_4', 'agric_trainings_5', 'agric_trainings_6']

HHLevel[AgricTr_Cols] = HHLevel[AgricTr_Cols].apply(pd.to_numeric, errors='coerce')

HHLevel['Agric Trainings'] = HHLevel[AgricTr_Cols].sum(axis=1)

HHLevel['Atleast 1 Agric Training'] = np.where(HHLevel['Agric Trainings'] > 0, 1,0)

In [43]:
WASHTr_Cols = ['wash_trainings_1', 'wash_trainings_2']

HHLevel[WASHTr_Cols] = HHLevel[WASHTr_Cols].apply(pd.to_numeric, errors='coerce')

HHLevel['WASH Trainings'] = HHLevel[AgricTr_Cols].sum(axis=1)

HHLevel['Atleast 1 WASH Training'] = np.where(HHLevel['WASH Trainings'] > 0, 1,0)

In [44]:
HTr_Cols = ['ht_trainings_1', 'ht_trainings_2', 'ht_trainings_3', 'ht_trainings_4']

HHLevel[HTr_Cols] = HHLevel[HTr_Cols].apply(pd.to_numeric, errors='coerce')

HHLevel['HR Trainings'] = HHLevel[AgricTr_Cols].sum(axis=1)

HHLevel['Atleast 1 HR Training'] = np.where(HHLevel['HR Trainings'] > 0, 1,0)

In [45]:
HORTr_Cols = ['hort_trainings_1', 'hort_trainings_2', 'hort_trainings_3', 'hort_trainings_4']

HHLevel[HORTr_Cols] = HHLevel[HORTr_Cols].apply(pd.to_numeric, errors='coerce')

HHLevel['HOR Trainings'] = HHLevel[AgricTr_Cols].sum(axis=1)

HHLevel['Atleast 1 HOR Training'] = np.where(HHLevel['HOR Trainings'] > 0, 1,0)

In [46]:
Cluster_Trainings = HHLevel.groupby(['district_1',  'cluster_t', 'ev_month'])[['Atleast 1 Agric Training', 
                                                                               'Atleast 1 WASH Training',
                                                                               'Atleast 1 HR Training',
                                                                               'Atleast 1 HOR Training',
                                                                              'ccdt_trainings_1']].mean(numeric_only = True).reset_index()
                                                                                          

In [47]:
Village_Trainings = HHLevel.groupby(['district_1',  'cluster_t', 'village_t', 'ev_month'])[['Atleast 1 Agric Training', 
                                                                               'Atleast 1 WASH Training',
                                                                               'Atleast 1 HR Training',
                                                                               'Atleast 1 HOR Training',
                                                                                           'ccdt_trainings_1']].mean(numeric_only = True).reset_index()
                                                                                          

In [48]:
Cluster_Trainings.to_excel('RWCluster_Trainings.xlsx', index = False)

In [49]:
Village_Trainings.to_excel('RWVillage_Trainings.xlsx', index = False)

In [50]:
RWComments = HHLevel[['district_1',  'cluster_t',  'village_t', 'ev_month', 'hh_comment',  'evaluators_comment']]

RWComments.to_excel('RWComments.xlsx', index = False)

In [46]:
RwandaJulySEs = {'Sheet1': Cluster_ScoresA, 'Sheet2': Village_ScoresA, 'Sheet3': Conclusions,
          'Sheet4': Village_NPS, 'Sheet5': Cluster_NPS, 'Sheet8': VLevel_SEs }

# Create a new Excel writer object
with pd.ExcelWriter('Rwanda_SEs.xlsx', engine='openpyxl') as writer:
    for sheet_name, df in JuneSEs.items():
        df.to_excel(writer, sheet_name=sheet_name, index=False)

print("Data saved successfully in 'RTV2024JuneSEs.xlsx'")

IndexError: At least one sheet must be visible